источник данных - Вакансии всех регионов России из ЕЦП «Работа в России» (https://trudvsem.ru/opendata/api)

### На какие вопросы можно ответить?

1. какое распределение вакансий по регионам? Абсолютные цифры и на душу населения
2. в каких регионах требуется больше специалистов с ученой степенью?
3. какое распределение вакансий по категориям и регионам?
4. средние зарплаты?
5. динамика зарплат по времени?
6. в каких категориях не указывают зарплаты?

и т.д.100500 вопросов можно придумать)

Также можно будет посмотреть исторические данные и попытаться ответить на вопрос, повлияло ли как-то то или иное событие на рынок вакансий.


### Что предлагается сделать?

Написать приложение, которое будет:
1. скачивать периодически данные из https://trudvsem.ru/opendata/api и сохранять в БД
2. визуализировать данные в BI системе (различные срезы, сегменты и по времени)

Инструменты:
1. clickhouse
2. airflow
3. grafana/datalens
4. python/jupyter notebook
5. docker

Загрузим для примера 10к записей

In [22]:
import requests
import pandas as pd


url = 'http://opendata.trudvsem.ru/api/v1/vacancies'
data = requests.get(url)
data = [vac['vacancy'] for vac in data.json()['results']['vacancies']]
data = pd.DataFrame.from_dict(data, orient='columns')
# adding company attributes
data['company_name'] = data['company'].apply(lambda x: x['name'], 1)
data['companycode'] = data['company'].apply(lambda x: x['companycode'], 1)
data['hr-agency'] = data['company'].apply(lambda x: x['hr-agency'], 1)
# adding region name
data['region_name'] = data['region'].apply(lambda x: x['name'], 1)
# adding specialization
data['specialisation'] = data['category'].apply(lambda x: x['specialisation'], 1)
data.head(3)

,id,source,region,company,creation-date,salary,salary_min,salary_max,job-name,vac_url,...,contact_person,work_places,code_profession,currency,term,company_name,companycode,hr-agency,region_name,specialisation
0,2be29788-0f4e-11ee-b544-0d972cff014f,Работодатель,"{'region_code': '3100000000000', 'name': 'Белг...","{'companycode': '1163668085570', 'email': 's_k...",2023-06-20,от 16500,16500,25000,продавец (с. Шаталовка),https://trudvsem.ru/vacancy/card/1163668085570...,...,Инна Олеговна,1,173534,«руб.»,NaN,"ООО ""ВОЗРОЖДЕНИЕ""",1163668085570,False,Белгородская область,"Продажи, закупки, снабжение, торговля"
1,e0c91fe3-edba-11ec-9f7d-bf2cfe8c828d,Работодатель,"{'region_code': '4200000000000', 'name': 'Кеме...","{'companycode': '1024201475000', 'email': 'nar...",2022-06-17,от 31000,31000,0,Фельдшер,https://trudvsem.ru/vacancy/card/1024201475000...,...,Матковская Наталья Александровна,1,273287,«руб.»,{'text': 'Социальный пакет'},ГБУЗ ННД,1024201475000,False,Кемеровская область - Кузбасс,Здравоохранение и социальное обеспечение
2,e0c10993-edba-11ec-9f7d-bf2cfe8c828d,Работодатель,"{'region_code': '4200000000000', 'name': 'Кеме...","{'companycode': '1024201475000', 'email': 'nar...",2022-06-17,от 26000,26000,0,Медицинская сестра палатная (постовая),https://trudvsem.ru/vacancy/card/1024201475000...,...,Матковская Наталья Александровна,3,240387,«руб.»,{'text': 'Социальный пакет'},ГБУЗ ННД,1024201475000,False,Кемеровская область - Кузбасс,Здравоохранение и социальное обеспечение


In [23]:
data.columns

Index(['id', 'source', 'region', 'company', 'creation-date', 'salary',
       'salary_min', 'salary_max', 'job-name', 'vac_url', 'employment',
       'schedule', 'duty', 'category', 'requirement', 'addresses',
       'social_protected', 'contact_list', 'contact_person', 'work_places',
       'code_profession', 'currency', 'term', 'company_name', 'companycode',
       'hr-agency', 'region_name', 'specialisation'],
      dtype='object')

In [25]:
data['specialisation'].value_counts()

Здравоохранение и социальное обеспечение                 42
Производство                                             19
Транспорт, автобизнес, логистика, склад, ВЭД             11
Образование, наука                                        8
Рабочие специальности                                     5
Продажи, закупки, снабжение, торговля                     4
Государственная служба, некоммерческие организации        3
Туризм, гостиницы, рестораны                              2
Банки, кредит, страхование, пенсионное обеспечение        1
Химическая, нефтехимическая, топливная промышленность     1
Легкая промышленность                                     1
Строительство, ремонт, стройматериалы, недвижимость       1
Кадровая служба, управление персоналом                    1
Пищевая промышленность                                    1
Name: specialisation, dtype: int64

In [26]:
data['employment'].value_counts()

Полная занятость       92
Временная               7
Частичная занятость     1
Name: employment, dtype: int64

In [28]:
data['company_name'].nunique()

18